In [ ]:
import psycopg2
import pandas as pd
import json


- Afficher pour chaque annexe, la liste des champs (./app/config.py)
- Pour chaque annexe, créer la requête pour récupérer les données concernant l'annexe de manière variabilisé
- Créer une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision. Cette fonction fait une requête à la base de donnée avec ces arguments. En retour elle renvoie un dataframe pandas 
- Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.
- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

# la liste des champs (./app/config.py) pour chaque annexe

In [ ]:
# 1- Les données de toutes les annexes
import app.backend.config as conf
data = conf.CHAMPS_ANNEXES
data

In [ ]:
for key, value in data.items() :
    print(key,value)

In [ ]:
for item in data:
    print(item)

In [63]:
print(data.keys())

dict_keys(['DATA_PRES_CROISEE', 'DATA_EMPRUNT', 'DATA_TRESORERIE', 'DATA_CHARGE', 'DATA_TIERS', 'DATA_CREDIT_BAIL', 'DATA_PPP', 'DATA_AUTRE_ENGAGEMENT', 'DATA_CONCOURS', 'DATA_RECETTE_AFFECTEE', 'DATA_FORMATION', 'DATA_FISCALITE', 'DATA_CONSOLIDATION', 'DATA_ORGANISME_ENG', 'DATA_ORGANISME_GROUP', 'DATA_PATRIMOINE', 'DATA_PERSONNEL', 'DATA_PERSONNEL_SOLDE', 'DATA_DETTE', 'DATA_VENTILATION', 'DATA_CONTRAT_COUV', 'DATA_AMORTISSEMENT_METHODE', 'DATA_PROVISION', 'DATA_APCP', 'DATA_SIGNATURE', 'DATA_SIGNATAIRE', 'DATA_ETAB_SERVICE', 'DATA_PRET', 'DATA_CONTRAT_COUV_REFERENCE', 'DATA_SERVICE_FERROVIAIRE_BUD', 'DATA_SERVICE_FERROVIAIRE_PATRIM', 'DATA_SERVICE_FERROVIAIRE_TER', 'DATA_FOND_COMM_HEBERGEMENT', 'DATA_FOND_EUROPEEN', 'DATA_FOND_EUROPEEN_PROGRAMMATION', 'DATA_FOND_AIDES_ECO', 'DATA_FORMATION_PRO_JEUNES', 'DATA_MEMBRESASA', 'DATA_FLUX_CROISES', 'DATA_SOMMAIRE'])


# 2 - Récupérer les données concernant l'annexe de manière variabilisé

In [65]:

def annexe(nom):
    donnees = data[nom]
    return donnees

In [ ]:
# 2.1 - Afficher les donnees de DATA_EMPRUNT
annexe(nom="DATA_EMPRUNT")

In [ ]:
# 2.2 - Afficher les donnees de toutes les annexes
for i in data:
    donnee = annexe(nom=i)
    print(donnee)

# Création d'une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision

In [ ]:
# 3- connexion à la base de donnée et  retour d'un dataframe pandas 
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")


def get_data(connexion):
    query= """SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = 'DATA_EMPRUNT'
    AND documentbudgetaire.exercice = '2020'
    AND documentbudgetaire.nature_dec = '09' 
        LIMIT 100 """
    dat = pd.read_sql_query(query, conn)
    # conn.close()
    return dat
get_data(connexion=conn)

In [ ]:
def connection_pg2():
    return psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

with connection_pg2() as conn:
    dat = get_data(conn)

dat.head(2)

In [ ]:
df_emprunt = get_data(connexion=conn)

In [ ]:
df_emprunt

In [ ]:
# 3.1 -
list(df_emprunt.columns)

In [ ]:
# 3.2 -
df_emprunt.json_annexe

In [ ]:
# 3.3 -
for item in df_emprunt.json_annexe:
    print(type(item))

In [71]:
# 3.4 - avec f string de manière variabiliser
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year, nature_dec):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{nature_dec}' 
        LIMIT 5 """
    dat = pd.read_sql_query(query, conn)
    return dat
get_data(connexion=conn, annexe='DATA_EMPRUNT', year=2019, nature_dec= '09')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,255693,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""95606.73""}]",44508,21700509900015,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",44508,CNE DE TROMAREY 22400,70509,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""65888"", ""ContNat"": ""65"", ""ArtSpe""...",21700509900015,21700509900015,COMMUNE DE TROMAREY,COMMUNE,70
1,256199,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""F"", ""...",44601,21560121200693,"[DATA_EMPRUNT, DATA_SIGNATURE, DATA_SOMMAIRE]",44601,BUDGET ZAC DU PERISTYLE,56121000000,M14-M14_COM_SUP3500,...,None,3,A,21560121200016,"[{""Nature"": ""6045"", ""Fonction"": ""824"", ""ContNa...",21560121200693,21560121200693,VILLE DE LORIENT,,None
2,256335,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",44626,21540065600017,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FISCALITE, ...",44626,Budget Commune,54065,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",21540065600017,21540065600017,Commune de BERTRICHAMPS,COMMUNE,54
3,256904,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""TxActuaInit"": ""0.000"", ...",44720,20004694400019,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_FISCALITE...",44720,SYNDICAT MIXTE DU MASSIF DES M,,M14-M14_COM_SUP3500,...,None,1,P,None,"[{""Nature"": ""6042"", ""LibCpte"": ""Achats de pres...",20004694400019,20004694400019,SYNDICAT MIXTE DU MASSIF DES MAURES,,None
4,257062,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",44749,24300065000032,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONSOLIDATION,...",44749,Budget Eau Potable,30003,M4-M49_D,...,None,1,A,24300065000016,"[{""Nature"": ""6811"", ""ContNat"": ""042"", ""ArtSpe""...",24300065000032,24300065000032,COMMUNAUTE DE COMMUNES TERRE DE CAMARGUE,Communauté de commune à DGF bonifiée,30


# Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.

In [75]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace          

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [78]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
print(next(replace_natdec))
print(next(replace_natdec))

('01', 'Budget primitif')
('02', 'Décision modificative')


In [79]:
list(df_emprunt.columns)

['id',
 'type_annexe',
 'json_annexe',
 'fk_id_document_budgetaire',
 'siret_etablissement',
 'list_annexes',
 'id',
 'libelle',
 'code_insee',
 'nomenclature',
 'exercice',
 'nature_dec',
 'num_dec',
 'nature_vote',
 'type_budget',
 'id_etabl_princ',
 'json_budget',
 'fk_siret_collectivite',
 'siret_coll',
 'libelle_collectivite',
 'nature_collectivite',
 'departement']

In [80]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(df_emprunt.columns) if item not in interesting_columns]
df = df_emprunt.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
df_min
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21830134900013,MAIRIE DE TARADEAU,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830134900013,MAIRIE DE TARADEAU,Commune
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""9393389.11""}]",20006755100214,EAU POTABLE,M4-M49_D,2020,Compte administratif,1,Annexe,20006755100214,THONON AGGLOMERATION,EPCI
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21570325700018,COMMUNE DE HILSPRICH,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21570325700018,COMMUNE DE HILSPRICH,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21270354000014,COMMUNE D'IVILLE,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21270354000014,COMMUNE D'IVILLE,Commune
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21870330400012,BUDGET COMMUNAL M14,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21870330400012,CHAMBORET,Coll.


In [81]:
df_min.json_annexe

0    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
1     [{"CodTypEmpr": "10", "RReelFon": "9393389.11"}]
2    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
3    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
4    [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...
Name: json_annexe, dtype: object

In [86]:
EMPRUNT_FIELD = annexe(nom="DATA_EMPRUNT")


In [ ]:
df.json_annexe

In [89]:
# extraction de la colonne json_annexe
dfs = []
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))

json_df.assign(**row)
#print('dfs')
#print(dfs)
df_to_csv = pd.concat(dfs)
df_to_csv

,CodTypEmpr,CodProfilAmort,CodProfilAmortDtVote,ProfilAmort,CodArticle,LibCpte,AnEncaisse,ObjEmpr,MtEmprOrig,DureeRest,...,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,01,X,X,Echéance constante,1641,Emprunts en euros,2005,ACQUISITION VILLA LEMEE,150000.00,0,...,21830134900013,MAIRIE DE TARADEAU,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830134900013,MAIRIE DE TARADEAU,Commune
1,06,X,X,Echéance constante,1641,Emprunts en euros,2011,REHABILITATION PLACE DE LA MAIRIE,200000.00,0,...,21830134900013,MAIRIE DE TARADEAU,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830134900013,MAIRIE DE TARADEAU,Commune
2,06,X,X,Echéance constante,1641,Emprunts en euros,2011,ACQUISITION FONCIERE PARCELLE AC 107 A L'ORMEA...,200000.00,0,...,21830134900013,MAIRIE DE TARADEAU,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830134900013,MAIRIE DE TARADEAU,Commune
3,01,C,C,None,1641,Emprunts en euros,2013,TRAVAUX DE VOIRIE,250000.00,2.5,...,21830134900013,MAIRIE DE TARADEAU,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830134900013,MAIRIE DE TARADEAU,Commune
4,01,C,C,None,1641,Emprunts en euros,2017,INVESTISSEMENTS 2017,600000.00,16.67,...,21830134900013,MAIRIE DE TARADEAU,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830134900013,MAIRIE DE TARADEAU,Commune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,01,P,P,None,1641,Emprunts en euros,2017,RUE DE LEGLISE ET RENOV. MAIRIE 10,300000.00,11,...,21540371800012,BUDGET COMMUNAL,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21540371800012,COMMUNE DE MOINEVILLE,COMMUNE
3,10,None,None,None,None,None,None,None,None,None,...,21540371800012,BUDGET COMMUNAL,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21540371800012,COMMUNE DE MOINEVILLE,COMMUNE
0,10,None,None,None,None,None,None,None,None,None,...,21440126700106,CELLULES COMMERCIALES,M4-M4,2020,Compte administratif,1,Annexe,21440126700106,COMMUNE LA PLAINE SUR MER,Commune
0,01,P,P,None,1641,None,2018,Emprunt pour le centre commercial,35000.00,13.04,...,21860254800032,CENTRE COMMERCIAL DE SAULGE,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21860254800032,CENTRE COMMERCIAL DE SAULGE,CENTRE COMMERCIAL DE SAULGE


- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

In [ ]:
def generate_csv(df, name):
    csv_field = df.to_csv(name)
    return csv_field
generate_csv(df_emprunt, "emprunt.csv")

In [85]:
# 3.4 - avec f string
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year, numero_docbudge):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{numero_docbudge}' 
        LIMIT 5 """
    dat = pd.read_sql_query(query, conn)
    return dat
get_data(connexion=conn, annexe='DATA_EMPRUNT', year=2019, numero_docbudge= '09')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,257062,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",44749,24300065000032,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONSOLIDATION,...",44749,Budget Eau Potable,30003,M4-M49_D,...,None,1,A,24300065000016,"[{""Nature"": ""6811"", ""ContNat"": ""042"", ""ArtSpe""...",24300065000032,24300065000032,COMMUNAUTE DE COMMUNES TERRE DE CAMARGUE,Communauté de commune à DGF bonifiée,30
1,257269,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",44787,24974007700048,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_CONCOURS,...",44787,C.I.VI.S.97410 SAINT-PIERRE,249740077,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""001"", ""Fonction"": ""020"", ""ContNat...",24974007700048,24974007700048,C.I.VI.S.,dont la population est de 3500 habitants et plus,None
2,258035,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""35810.15""}]",44924,21700234400059,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",44924,Forêt FILAIN 60400,None,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",21700234400059,21700234400059,Forêt FILAIN,COMMUNE,70
3,258902,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",45070,20003902200096,"[DATA_EMPRUNT, DATA_CREDIT_BAIL, DATA_CONSOLID...",45070,ATELIERS RELAIS 12108,35360,M14-M14_COM_SUP3500,...,None,3,A,20003902200013,"[{""Nature"": ""001"", ""Fonction"": ""90"", ""ContNat""...",20003902200096,20003902200096,VITRE COMMUNAUTE,COMMUNE dont la population est de 3500 habitan...,None
4,259147,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",45109,20005480700314,"[DATA_EMPRUNT, DATA_ORGANISME_ENG, DATA_PERSON...",45109,CT5 EAU POTABLE,None,M4-M49_D,...,None,1,A,20005480700017,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",20005480700314,20005480700314,Métropole D' Aix-Marseille-Provence,EPCI,None


In [ ]:
annexe_dic = dict()
for item in data:
    annexe_dic[item] =  get_data(connexion=conn, annexe=item, year=2020, numero_docbudge= '09')

In [ ]:
annexe_dic["DATA_EMPRUNT"]

In [ ]:
for item in annexe_dic:
    print(annexe_dic[item])

In [ ]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres", 
                        port = "5432")
query = """
 SELECT * from annexe 
 JOIN documentbudgetaire 
     ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
 JOIN collectivite 
     ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
 WHERE annexe.type_annexe = 'DATA_EMPRUNT' 
 AND documentbudgetaire.exercice = '2020'
 AND documentbudgetaire.nature_dec = '09' 
 LIMIT 100
 """
dat = pd.read_sql_query(query, conn)
conn.close()



In [ ]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace

In [ ]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(dat.columns) if item not in interesting_columns]
df = dat.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

In [ ]:
df_min

In [ ]:
dfs = []
#EMPRUNT_FIELD = interesting_columns
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
    print(json_df)
    #json_df.assign(**row)

In [ ]:
df_to_csv = pd.concat(dfs)
df_to_csv.to_csv('./csv/2020-CA-Limit100.csv', index=False)  

In [ ]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']